In [3]:
import torch
import retro
import gym
import torch
import torch.nn as nn

# import time to slow down the game
import time


# import env baseclass
from gym import Env
# import space shapes for the 
from gym.spaces import MultiBinary, Box
# import numpy to calculate change
import numpy as np
# for grayscale
import cv2
# for plotting
import matplotlib.pyplot as plt

# train and tune and the same time
# optimization framework
import optuna
import tqdm as notebook_tqdm
from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.evaluation import evaluate_policy

# import sb3 monitor for logging
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, SubprocVecEnv
import os


import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import BasePolicy
from typing import Tuple

# game='SuperMarioWorld-Snes'


from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import VecMonitor
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import os

import gym
import numpy as np
# from RandomAgent import TimeLimitWrapper

# Train

In [5]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True

In [6]:
def make_env(env_id,rank,seed=0):
    def _init():
        env= retro.make(game=env_id)
        env = MaxAndSkipEnv(env,4)
        env.seed(seed+rank)
        return env
    
    set_random_seed(seed)
    return _init

log_dir="tmp/"
os.makedirs(log_dir,exist_ok=True)

In [7]:
env_id = "SuperMarioBros-Nes"

In [8]:
num_cpu = 4

In [9]:
env = VecMonitor(SubprocVecEnv([make_env(env_id,i) for i in range(num_cpu)]))

In [10]:
model = PPO('CnnPolicy',env,verbose=1,tensorboard_log='./board/',learning_rate=0.00003)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [15]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [11]:
# callback = SaveOnBestTrainingRewardCallback(check_freq=1000,log_dir=log_dir)

In [16]:
CHECKPOINT_DIR = './mario_new/neww/'

In [17]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)


In [19]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./board/PPO_3
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 663       |
|    ep_rew_mean     | 1754.6364 |
| time/              |           |
|    fps             | 578       |
|    iterations      | 1         |
|    time_elapsed    | 14        |
|    total_timesteps | 8192      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 707         |
|    ep_rew_mean          | 1932.9524   |
| time/                   |             |
|    fps                  | 202         |
|    iterations           | 2           |
|    time_elapsed         | 80          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.053770423 |
|    clip_fraction        | 0.463       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.47       |
|    explained_variance   | 0.843    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 718         |
|    ep_rew_mean          | 2038.34     |
| time/                   |             |
|    fps                  | 134         |
|    iterations           | 11          |
|    time_elapsed         | 669         |
|    total_timesteps      | 90112       |
| train/                  |             |
|    approx_kl            | 0.057106398 |
|    clip_fraction        | 0.473       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.98       |
|    explained_variance   | 0.894       |
|    learning_rate        | 3e-05       |
|    loss                 | 613         |
|    n_updates            | 504         |
|    policy_gradient_loss | 0.0179      |
|    value_loss           | 1.63e+03    |
-----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 730       

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 899       |
|    ep_rew_mean          | 2080.21   |
| time/                   |           |
|    fps                  | 128       |
|    iterations           | 21        |
|    time_elapsed         | 1337      |
|    total_timesteps      | 172032    |
| train/                  |           |
|    approx_kl            | 0.0708328 |
|    clip_fraction        | 0.483     |
|    clip_range           | 0.2       |
|    entropy_loss         | -3.65     |
|    explained_variance   | 0.898     |
|    learning_rate        | 3e-05     |
|    loss                 | 544       |
|    n_updates            | 604       |
|    policy_gradient_loss | 0.0231    |
|    value_loss           | 1.91e+03  |
---------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 945        |
|    ep_rew_mean          | 2113.4   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.04e+03   |
|    ep_rew_mean          | 2223.79    |
| time/                   |            |
|    fps                  | 131        |
|    iterations           | 31         |
|    time_elapsed         | 1925       |
|    total_timesteps      | 253952     |
| train/                  |            |
|    approx_kl            | 0.08843322 |
|    clip_fraction        | 0.476      |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.95      |
|    explained_variance   | 0.941      |
|    learning_rate        | 3e-05      |
|    loss                 | 829        |
|    n_updates            | 704        |
|    policy_gradient_loss | 0.0197     |
|    value_loss           | 1.44e+03   |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.06e+03   |
|    ep_rew_mean

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1.14e+03 |
|    ep_rew_mean          | 2165.07  |
| time/                   |          |
|    fps                  | 132      |
|    iterations           | 41       |
|    time_elapsed         | 2529     |
|    total_timesteps      | 335872   |
| train/                  |          |
|    approx_kl            | 1.560473 |
|    clip_fraction        | 0.536    |
|    clip_range           | 0.2      |
|    entropy_loss         | -2.19    |
|    explained_variance   | 0.951    |
|    learning_rate        | 3e-05    |
|    loss                 | 96       |
|    n_updates            | 804      |
|    policy_gradient_loss | -0.0504  |
|    value_loss           | 128      |
--------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.11e+03  |
|    ep_rew_mean          | 2092.11   |
| time/              

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.14e+03   |
|    ep_rew_mean          | 2133.05    |
| time/                   |            |
|    fps                  | 133        |
|    iterations           | 81         |
|    time_elapsed         | 4956       |
|    total_timesteps      | 663552     |
| train/                  |            |
|    approx_kl            | 0.22488545 |
|    clip_fraction        | 0.366      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.46      |
|    explained_variance   | 0.952      |
|    learning_rate        | 3e-05      |
|    loss                 | 1.37e+03   |
|    n_updates            | 1204       |
|    policy_gradient_loss | 0.0464     |
|    value_loss           | 688        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.16e+03   |
|    ep_rew_mean

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.19e+03  |
|    ep_rew_mean          | 2219.77   |
| time/                   |           |
|    fps                  | 133       |
|    iterations           | 91        |
|    time_elapsed         | 5579      |
|    total_timesteps      | 745472    |
| train/                  |           |
|    approx_kl            | 0.0832528 |
|    clip_fraction        | 0.304     |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.62     |
|    explained_variance   | 0.957     |
|    learning_rate        | 3e-05     |
|    loss                 | 392       |
|    n_updates            | 1304      |
|    policy_gradient_loss | 0.028     |
|    value_loss           | 1.1e+03   |
---------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.21e+03    |
|    ep_rew_mean          | 2241.8

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.14e+03   |
|    ep_rew_mean          | 2193.33    |
| time/                   |            |
|    fps                  | 134        |
|    iterations           | 101        |
|    time_elapsed         | 6173       |
|    total_timesteps      | 827392     |
| train/                  |            |
|    approx_kl            | 0.10086771 |
|    clip_fraction        | 0.232      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.887     |
|    explained_variance   | 0.986      |
|    learning_rate        | 3e-05      |
|    loss                 | 55.7       |
|    n_updates            | 1404       |
|    policy_gradient_loss | 0.0148     |
|    value_loss           | 280        |
----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.08e+03  |
|    ep_rew_mean   

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.74e+03   |
|    ep_rew_mean          | 2004.32    |
| time/                   |            |
|    fps                  | 133        |
|    iterations           | 111        |
|    time_elapsed         | 6827       |
|    total_timesteps      | 909312     |
| train/                  |            |
|    approx_kl            | 0.05783988 |
|    clip_fraction        | 0.315      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.59      |
|    explained_variance   | 0.954      |
|    learning_rate        | 3e-05      |
|    loss                 | 865        |
|    n_updates            | 1504       |
|    policy_gradient_loss | 0.0222     |
|    value_loss           | 1.51e+03   |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.64e+03   |
|    ep_rew_mean

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1.6e+03  |
|    ep_rew_mean          | 1979.18  |
| time/                   |          |
|    fps                  | 132      |
|    iterations           | 121      |
|    time_elapsed         | 7455     |
|    total_timesteps      | 991232   |
| train/                  |          |
|    approx_kl            | 6.971355 |
|    clip_fraction        | 0.651    |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.758   |
|    explained_variance   | 0.948    |
|    learning_rate        | 3e-05    |
|    loss                 | 68.3     |
|    n_updates            | 1604     |
|    policy_gradient_loss | -0.0134  |
|    value_loss           | 220      |
--------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.63e+03    |
|    ep_rew_mean          | 1967.75     |
| time/      

In [20]:
model.save(env_id)

In [21]:
env_id

'SuperMarioBros-Nes'

In [22]:
model.save("best_model.zip")